<a href="https://colab.research.google.com/github/CCS-ZCU/avarice/blob/master/scripts/GreLa_API_getting_started_passau2026_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# requirements
import requests
import pandas as pd
pd.set_option("display.max_columns", None)

## Introduction

The database is publicly available online from the following endpoint:

```bash
https://ccs-lab.zcu.cz/grela-api/api/query
```

Through the API, you can query it using the same queries as the local version.

For instance, to retrieve the first 10 rows from  the `works` table, your SQL query would be:
```
SELECT * FROM works LIMIT 10;
```

To execute the query from the command line, you could use the `curl` command, e.g.:
```bash
curl -X POST https://ccs-lab.zcu.cz/grela-api/api/query \
  -H "Content-Type: application/json" \
  -d '{"query": "SELECT * FROM works", "format": "json"}'
```



The API returns the response as a JSON object.

The query output is available as a downloadable file, which you can download using the `download_url` field in the response, which you can load directly into Python as a pandas DataFrame object.

In Python, the whole pipeline consists of several steps:

In [ ]:
# (1) define the database connection
api_url = "https://ccs-lab.zcu.cz/grela-api/api/query"
# (2) define the query, e.g. the following one to obtain all work-level metadata
query = "SELECT * FROM works"
# (3) execute the query via requests
response = requests.post(api_url, json={"query": query})
# (4) check the response - [200] indicates that everything works properly
response

<Response [200]>

In [ ]:
response.json()["download_url"]

'https://ccs-lab.zcu.cz/grela-api-out/a6885169-6269-4341-acd7-9ac5f4dab252.parquet'

In [ ]:
# (5) retrieve the download URL from the response with error checking
download_url = response.json()["download_url"]
print(download_url)

https://ccs-lab.zcu.cz/grela-api-out/a6885169-6269-4341-acd7-9ac5f4dab252.parquet


In [ ]:
# (6) load the file object into a pandas DataFrame
df = pd.read_parquet(download_url)
len(df)

11127

In [ ]:
df.head(5)

,grela_source,grela_id,author,title,not_before,not_after,date_random,provenience,place_publication,place_geonames,author_viaf,author_wd,author_gnd,title_viaf,subcorpus_specific_metadata,token_count,sentence_count
0,lagt,lagt_ggm0001.ggm001,Anonymous,Anametresis Pontou,1,400,157,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",422,23
1,lagt,lagt_ogl0001.ogl001,Pinytus,De Epistola Pinyti ad Dionysium,101,200,174,christian,None,None,None,None,None,None,"{""lagt_tlg_epithet"":""[]"",""lagt_genre"":""[]"",""no...",81,3
2,lagt,lagt_pta0001.pta001,Severian of Gabala,De fide et lege naturae,400,409,403,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",4495,327
3,lagt,lagt_pta0001.pta002,Severian of Gabala,De paenitentia et compunctione,400,409,406,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",6987,539
4,lagt,lagt_pta0001.pta003,Severian of Gabala,In ascensionem domini nostri Iesu Christi et i...,400,409,406,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",12070,1041


## Working with the `works` table

the `works` table contains all the metadata about the works in the database.

In [ ]:
# you can combain all previous steps into 2-liner, or perhaps even 1-liner, if you wish...
# and obtain the entire `works` table as a pandas DataFrame
query = "SELECT * FROM works"
df = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])

In [ ]:
# you can also select specific works, for instance, based on the date columns:
query = """
SELECT * FROM works as w
WHERE (w.not_before > 1050 AND w.not_before < 1250)
   OR (w.not_after > 1050 AND w.not_after < 1250);
"""

df = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])
df.head(5)

,grela_source,grela_id,author,title,not_before,not_after,date_random,provenience,place_publication,place_geonames,author_viaf,author_wd,author_gnd,title_viaf,subcorpus_specific_metadata,token_count,sentence_count
0,lagt,lagt_tlg2703.tlg001,Anna Comnena,Alexias,1083,1153,1094,None,None,None,111167713,Q179284,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",164519,6485
1,lagt,lagt_tlg3135.tlg001,Joannes Zonaras,Epitome Historiarum,1081,1118,1102,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",392116,17683
2,lagt,lagt_tlg3135.tlg011,Joannes Zonaras,Argumenta,1081,1118,1084,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",27443,2607
3,lagt,lagt_tlg3135.tlg012,Joannes Zonaras,Chronologia,1081,1118,1096,None,None,None,18151776766218010762,Q32052,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",1133,22
4,lagt,lagt_tlg4026.tlg001,Anonymi in Aristotelis Artem Rhetoricam,In Aristotelis artem rhetoricam commentarium,601,1200,891,None,None,None,None,None,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",108762,5147


In [ ]:

df = df[df["grela_source"]=="cc"]

In [ ]:
len(df)

1590

In [ ]:
df["token_count"].sum()

np.int64(83517461)

## Working with the `sentences` table

In [ ]:
# extract a subset of sentences with work level metadata, select on specific date
query = """
        SELECT s.grela_id,
               s.sentence_id,
               s.sent_text,
               w.*
        FROM sentences s
                 JOIN works w ON s.grela_id = w.grela_id
        WHERE (w.not_before > 1050 AND w.not_before < 1100) OR (w.not_after > 1050 AND w.not_after < 1100);
        """

sentences_1050_1100 = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])

In [ ]:
len(sentences_1050_1100)

1301269

In [ ]:
text = " ".join(sentences_1050_1100["sent_text"].tolist())
text[:10000]

"Praefatio #editionis Bituricensis anni@# 1690. Les Assises de Hierusalem sont les loix, statuts, usages et coutumes accordees au royaume de Hierusalem par Godefroy duc de Buillon l' an 1099, par l' auis du patriarche et de ses barons, apres qu' il en eut ete elu roy. Elles sont appellees Assises, parce qu' elles furent faites en l' assise ou assemblee des grands du royaume, de meme que la celebre ordonnance de Godefroy comte de Bretagne et de ses barons de l' an 1185, pour le partage noble, est appellee l' Assise du comte Godefroy. Et chez Houeden: #assisae Henrici regis factae apud Clarendonum@# dans le #regiam majestatem@# lib. Iii., c. 27, SS 2., #ex beneficio Constitutionis regni, quae Assisa nominatur.@# L' assise du roy Bauduin ch. 117. L' assise du roy Amaury ch. 273. Assise du roy Amaury et du roy Bauduin son fils. Ass. ch. 288. Assise du roy Hugues ch. 309. Spelman. f. 53. #in uerbo@# Assisia. Ces Assises furent ecrites par l' ordre de Godefroy de Buillon, et scellees de son 

In [ ]:

sentences_1050_1100.head(10)

,grela_id,sentence_id,sent_text,grela_source,grela_id_1,author,title,not_before,not_after,date_random,provenience,place_publication,place_geonames,author_viaf,author_wd,author_gnd,title_viaf,subcorpus_specific_metadata,token_count,sentence_count
0,cc_10532,cc_10532_0,Praefatio #editionis Bituricensis anni@# 1690.,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
1,cc_10532,cc_10532_1,"Les Assises de Hierusalem sont les loix, statu...",cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
2,cc_10532,cc_10532_2,"Elles sont appellees Assises, parce qu' elles ...",cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
3,cc_10532,cc_10532_3,Et chez Houeden:,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
4,cc_10532,cc_10532_4,#assisae Henrici regis factae apud Clarendonum...,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
5,cc_10532,cc_10532_5,"Iii., c. 27, SS 2., #ex beneficio Constitution...",cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
6,cc_10532,cc_10532_6,Assise du roy Amaury et du roy Bauduin son fils.,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
7,cc_10532,cc_10532_7,Ass.,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
8,cc_10532,cc_10532_8,ch. 288.,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542
9,cc_10532,cc_10532_9,Assise du roy Hugues ch. 309.,cc,cc_10532,Godefridus Bullonius,Assises de Jerusalem,1080,1080,1080,None,None,None,13555210,,None,None,"{""lagt_tlg_epithet"":null,""lagt_genre"":null,""no...",19074,542


In [ ]:
# Extract token data from all sentences containing the lemma "liber" with the POS tag "NOUN"
# include also the morphological data for each token within the sentence

query = """
WITH target_matches AS (
    SELECT sentence_id
    FROM tokens t
    WHERE t.lemma = 'auaritia' AND t.pos IN ('NOUN')
)

SELECT
    sentence_id,
    LIST(
        STRUCT_PACK(
            token_id    := token_id,
            token_text  := token_text,
            lemma       := lemma,
            pos         := pos,
            char_start  := char_start,
            char_end    := char_end,
            sentence_id := sentence_id
        )
        ORDER BY token_id
    ) AS tokens
FROM tokens
WHERE sentence_id IN (SELECT DISTINCT sentence_id FROM target_matches)
GROUP BY sentence_id;
"""

target_sentences_token_data  = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])
target_sentences_token_data.head(5)

,sentence_id,tokens
0,cc_13981_11503,"[{'char_end': 3, 'char_start': 0, 'lemma': 'it..."
1,cc_13981_13314,"[{'char_end': 4, 'char_start': 0, 'lemma': 'nu..."
2,cc_13099_148,"[{'char_end': 9, 'char_start': 0, 'lemma': 'ar..."
3,cc_15571_568,"[{'char_end': 10, 'char_start': 0, 'lemma': 'l..."
4,cc_20328_209,"[{'char_end': 2, 'char_start': 0, 'lemma': 'si..."


In [ ]:
len(target_sentences_token_data)

16477

In [ ]:
[t["lemma"] for t in target_sentences_token_data["tokens"].tolist()[0] if t["pos"] in ["NOUN", "VERB", "ADJ"]]

['lupa',
 'uir',
 'omnis',
 'bonus',
 '-',
 'mitto',
 'exilium',
 'dico',
 'prudentia',
 'auaritia',
 'ostendeo',
 'magnus',
 'pecunia',
 'salus',
 'pecunia',
 'plus',
 'facio',
 'conuenio']

### Working with the `tokens` table

We can get all tokens data based on a specific work attribute, e.g. author name pattern

In the example below, we select all tokens belonging to works by Plato.

The tokens are in the order as they appear in the text, so you use this output to compile the text.

You can also filter the tokens based on their POS tags, e.g. nouns, adjectives, etc.

We also keep track of the `sentence_id`, so you can at any time retrieve the raw text of the sentence text from the `sentences` table.

In [ ]:
query = """
    SELECT t.*, w.*
    FROM tokens t
    JOIN works w ON t.grela_id = w.grela_id
    WHERE w.author LIKE 'Plato'
"""

plato_tokens = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])
len(plato_tokens)

1056570

You can also retrieve the token data for a subset of works, like for all Christian works from the first three centuries

In [ ]:
### all token data from sentences with lagt_provenience "christian" and "not_ and not_after intersecting the first three centuries

query = """
SELECT
    t.sentence_id,
    w.author,
    w.title,
    w.not_before,
    w.not_after,
    LIST(
        STRUCT_PACK(
            token_id    := t.token_id,
            token_text  := t.token_text,
            lemma       := t.lemma,
            pos         := t.pos,
            char_start  := t.char_start,
            char_end    := t.char_end,
            sentence_id := t.sentence_id
        )
        ORDER BY t.token_id
    ) AS tokens
FROM tokens t
JOIN works w ON t.grela_id = w.grela_id
WHERE w.provenience = 'christian'
  AND w.not_before >= 0 AND w.not_before < 300
  AND w.not_after  >= 0 AND w.not_after  < 300
GROUP BY t.sentence_id, w.author, w.title, w.not_before, w.not_after
"""

EC_sentence_tokens = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])
len(EC_sentence_tokens)

122638

From this output you can easily get to the data format we used previously in the LAGT dataset and elsewhere for training distributional semantic models.

In [ ]:
EC_sentence_tokens["lemmatized_sentence"] = EC_sentence_tokens["tokens"].apply(lambda x: [t["lemma"] for t in x if t["pos"] in ["NOUN", "VERB", "ADJ", "PROPN"]])
EC_sentence_tokens["lemmatized_sentence"].tolist()[:10]

[['ἀνδρίζω', 'Ἑρμῆς'],
 ['ἥκω', 'ἐθέλω'],
 ['κυρία', 'ἀφίημι', 'πρέσβυς', 'πρῶτος', 'καθίζω'],
 ['φημί',
  'κυρία',
  'ἀδελφός',
  'ἀναγγέλλω',
  'ἱλαρός',
  'γίγνομαι',
  'ἀκούω',
  'γιγνώσκω',
  'κύριος',
  'πολύς',
  'δόξα'],
 ['ἀποκάλυψις', 'τέλος', 'ἔχω'],
 ['λέγω'],
 ['ἀποκρίνω', 'λέγω'],
 ['συμφωνέω', 'εἰρήνη', 'ἔχω', 'ἀκούω'],
 ['οἰκοδομή',
  'ὑπάγω',
  'λατομέω',
  'κύριος',
  'δοκιμάζω',
  'πορεύω',
  'εὐθύτης',
  'κύριος',
  'κατορθόω',
  'ἐντολή'],
 ['πρῶτος', 'γιγνώσκω']]

## Combined context query

You can ultimately run very complex queries using the API.

In the example below, we retrieve the contextual data for all tokens matching the lemma "mercurius" with the POS tag "NOUN".

For each token, we retrieve:
- (1) the raw sentence in which the (lemmatized) token appears (`semtence_tokens`),
- (2) the token data from the sentence in which the token appears (`sentence_tokens`),
- (3) a broader context, including also 1 raw sentence before and 1 raw sentence after the target token (`context_3sents`)
- (4) concordance from the 10 tokens preceding and 10 tokens following the target token (i.e. 10+1+10 tokens) (`concordance_tokens`).

All these co-occurrence data are then returned within a single table and can be used for different kinds of co-occurrence and semantic analyses.

In [ ]:


def get_grouped_token_data_with_metadata_api(api_url: str, lemma: str, pos_tags: list) -> pd.DataFrame:
    """
    API version of grouped token query with context and metadata.
    """
    pos_placeholder = ", ".join(f"'{tag}'" for tag in pos_tags)
    lemma_literal = lemma.replace("'", "''")  # SQL-safe single quotes

    query = f"""
    WITH target_matches AS (
        SELECT
            t.sentence_id,
            t.grela_id,
            s.position AS sentence_position,
            t.token_id AS target_token_id,
            t.char_start AS target_char_start,
            t.char_end AS target_char_end
        FROM tokens t
        JOIN sentences s USING (sentence_id)
        WHERE t.lemma = '{lemma_literal}'
        AND t.pos IN ({pos_placeholder})
    ),

    context_3sents AS (
        SELECT
            tm.sentence_id,
            STRING_AGG(s.sent_text, ' | ' ORDER BY s.position) AS context_3sents
        FROM target_matches tm
        JOIN sentences s
          ON s.grela_id = tm.grela_id
         AND s.position BETWEEN tm.sentence_position - 1
                            AND tm.sentence_position + 1
        GROUP BY tm.sentence_id
    ),

    sentence_tokens AS (
        SELECT
            sentence_id,
            LIST(
              STRUCT_PACK(
                 token_id := token_id,
                 token_text := token_text,
                 lemma := lemma,
                 pos := pos,
                 char_start := char_start,
                 char_end := char_end,
                 sentence_id := sentence_id
              )
              ORDER BY token_id
            ) AS tokens
        FROM tokens
        WHERE sentence_id IN (SELECT DISTINCT sentence_id FROM target_matches)
        GROUP BY sentence_id
    ),

    concordance_tokens AS (
        SELECT
            tm.sentence_id,
            tm.target_token_id,
            LIST(
              STRUCT_PACK(
                 token_id := ct.token_id,
                 token_text := ct.token_text,
                 lemma := ct.lemma,
                 pos := ct.pos,
                 char_start := ct.char_start,
                 char_end := ct.char_end,
                 sentence_id := ct.sentence_id
              )
              ORDER BY ct.token_id
            ) AS concordance_tokens
        FROM target_matches tm
        JOIN tokens ct
          ON ct.grela_id = tm.grela_id
         AND ct.token_id BETWEEN tm.target_token_id - 10
                            AND tm.target_token_id + 10
        GROUP BY tm.sentence_id, tm.target_token_id
    )

    SELECT
        w.author,
        w.title,
        tm.grela_id,
        tm.sentence_id,
        s.sent_text AS sentence_text,
        c3.context_3sents,
        st.tokens,
        ct.concordance_tokens,
        w.not_before,
        w.not_after,
        w.date_random,
        tm.target_token_id,
        tm.target_char_start,
        tm.target_char_end
    FROM target_matches tm
    JOIN sentences s USING (sentence_id)
    JOIN works w ON w.grela_id = tm.grela_id
    LEFT JOIN context_3sents c3 USING (sentence_id)
    LEFT JOIN sentence_tokens st USING (sentence_id)
    LEFT JOIN concordance_tokens ct
           ON ct.sentence_id = tm.sentence_id
          AND ct.target_token_id = tm.target_token_id
    ORDER BY tm.sentence_id, tm.target_token_id;
    """

    # Send query to the API
    response = requests.post(api_url, json={"query": query})
    response.raise_for_status()

    # Load the result
    download_url = response.json()["download_url"]
    return pd.read_parquet(download_url)

In [ ]:
# Example usage
term_contextual_data = get_grouped_token_data_with_metadata_api(
    api_url="https://ccs-lab.zcu.cz/grela-api/api/query",
    lemma="auaritia",
    pos_tags=["NOUN"])

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://ccs-lab.zcu.cz/grela-api/api/query

In [ ]:
term_contextual_data

,author,title,grela_id,sentence_id,sentence_text,context_3sents,tokens,concordance_tokens,not_before,not_after,date_random,target_token_id,target_char_start,target_char_end
0,"Van Helmont, Jan Baptist",Ortus medicinae,emlap_100100,emlap_100100_5113,"signum est haec mala prouenire ex sordibus, in...",non autem tremuisset terra. | signum est haec ...,"[{'char_end': 6, 'char_start': 0, 'lemma': 'si...","[{'char_end': 15, 'char_start': 11, 'lemma': '...",1648,1648,1648,259455820,69,77
1,"Helmont, Jan Baptista van","Ortus medicinae, id est, initia physicae inaud...",noscemus_688159,noscemus_688159_5678,"signum est haec mala prouenire ex sordibus, in...","Quae si contingant cum ortu haeresiarcharum, s...","[{'char_end': 6, 'char_start': 0, 'lemma': 'si...","[{'char_end': 15, 'char_start': 11, 'lemma': '...",1648,1648,1648,304615824,69,77
2,"Helmont, Jan Baptista van","Ortus medicinae, id est, initia physicae inaud...",noscemus_688159,noscemus_688159_6536,"signum est haec mala prouenire ex sordibus, in...","Quae si contingant cum ortu haeressarcharum, s...","[{'char_end': 6, 'char_start': 0, 'lemma': 'si...","[{'char_end': 15, 'char_start': 11, 'lemma': '...",1648,1648,1648,304629455,69,77


In [ ]:
# Example usage
labyrinthus_contextual_data = get_grouped_token_data_with_metadata_api(
    api_url="https://ccs-lab.zcu.cz/grela-api/api/query",
    lemma="λαβύρινθος",
    pos_tags=["NOUN", "PROPN", "ADJ"])

In [ ]:
labyrinthus_contextual_data#%%

,author,title,grela_id,sentence_id,sentence_text,context_3sents,tokens,concordance_tokens,not_before,not_after,date_random,target_token_id,target_char_start,target_char_end
0,Theocritus,Εἰδύλλια,lagt_tlg0005.tlg001,lagt_tlg0005.tlg001_1181,ἐγγύθι δ αὐτοῖν κεῖτο τὰ ταῖν χειροῖν ἀθλήματα...,"ἰχθύος ἀγρευτῆρες ὅμως δύο κεῖντο γέροντες, στ...","[{'char_end': 6, 'char_start': 0, 'lemma': 'ἐγ...","[{'char_end': 89, 'char_start': 80, 'lemma': '...",-299,-259,-268,1545279,142,152
1,Plutarch,Θησεύς,lagt_tlg0007.tlg001,lagt_tlg0007.tlg001_134,"ἐπεὶ δὲ κατέπλευσεν εἰς Κρήτην, ὡς μὲν οἱ πολλ...",διὸ καὶ καλεῖσθαι τὴν θεὸν Ἐπιτραγίαν. | ἐπεὶ ...,"[{'char_end': 4, 'char_start': 0, 'lemma': 'ἐπ...","[{'char_end': 98, 'char_start': 88, 'lemma': '...",96,116,97,34937189,141,151
2,Herodotus,Ἱστορίαι,lagt_tlg0016.tlg001,lagt_tlg0016.tlg001_2625,"καὶ δή σφι μνημόσυνα ἔδοξε λιπέσθαι κοινῇ, δόξ...",ἐς γὰρ δὴ τὰ πάντα ἱρὰ συνελέγοντο. | καὶ δή σ...,"[{'char_end': 3, 'char_start': 0, 'lemma': 'κα...","[{'char_end': 10, 'char_start': 7, 'lemma': 'σ...",-429,-424,-424,32769902,67,77
3,Herodotus,Ἱστορίαι,lagt_tlg0016.tlg001,lagt_tlg0016.tlg001_2627,εἰ γάρ τις τὰ ἐξ Ἑλλήνων τείχεά τε καὶ ἔργων ἀ...,τὸν ἐγὼ ἤδη εἶδον λόγου μέζω. | εἰ γάρ τις τὰ ...,"[{'char_end': 2, 'char_start': 0, 'lemma': 'εἰ...","[{'char_end': 67, 'char_start': 66, 'lemma': '...",-429,-424,-424,32769947,119,129
4,Herodotus,Ἱστορίαι,lagt_tlg0016.tlg001,lagt_tlg0016.tlg001_2629,"ἦσαν μέν νυν καὶ αἱ πυραμίδες λόγου μέζονες, κ...",καίτοι ἀξιόλογός γε καὶ ὁ ἐν Ἐφέσῳ ἐστὶ νηὸς κ...,"[{'char_end': 4, 'char_start': 0, 'lemma': 'εἰ...","[{'char_end': 69, 'char_start': 63, 'lemma': '...",-429,-424,-424,32769986,115,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Scholia in Lucianum,Scholia in Lucianum,lagt_tlg5029.tlg001,lagt_tlg5029.tlg001_4642,Ε λαβύρινθον κτλ.,"αὕτη γὰρ καὶ πρὸς τὸ εἶναι ἡμᾶς αἰτιᾶται, καθὰ...","[{'char_end': 2, 'char_start': 1, 'lemma': 'γε...","[{'char_end': 127, 'char_start': 120, 'lemma':...",301,400,336,14064091,3,13
96,Scholia in Lucianum,Scholia in Lucianum,lagt_tlg5029.tlg001,lagt_tlg5029.tlg001_601,λαβύρινθος ἦν οἴκημα σκολιώτατον·,ὡς πολλάκις παρεθέμεθα. | λαβύρινθος ἦν οἴκημ...,"[{'char_end': 11, 'char_start': 1, 'lemma': 'λ...","[{'char_end': 51, 'char_start': 45, 'lemma': '...",301,400,336,14017088,1,11
97,Scholia in Lucianum,Scholia in Lucianum,lagt_tlg5029.tlg001,lagt_tlg5029.tlg001_602,"καὶ ἐπεὶ σκολιὸς ἦν ἐν ταῖς ζητήσεσι, διὰ τοῦτ...",λαβύρινθος ἦν οἴκημα σκολιώτατον· | καὶ ἐπεὶ ...,"[{'char_end': 3, 'char_start': 0, 'lemma': 'κα...","[{'char_end': 8, 'char_start': 4, 'lemma': 'ἐπ...",301,400,336,14017104,57,67
98,Scholia in Pindarum,Scholia in Pindarum Nemean Odes,lagt_tlg5034.tlg001c,lagt_tlg5034.tlg001c_1609,ἡνίκα γὰρ ἐν τῷ Λαβυρίνθῳ ἐξησθένει πρὸς τὴν ἰ...,"τοῦτο δέ φησιν, ἐπειδὴ πρότερον ὁ Θησεὺς Ἀθηνα...","[{'char_end': 5, 'char_start': 0, 'lemma': 'ἡν...","[{'char_end': 52, 'char_start': 50, 'lemma': '...",-322,-30,-76,24587650,16,25
